# Create Database from Heatpump Keymark
## 1) download from https://keymark.eu/en/products/heatpumps/certified-products
## 2) use dash-skript to convert pdf into txt
## 3) use this skript:




In [1]:
import hplib
import simulate

In [2]:
# Import keymark data and save to csv database
hplib.ImportKeymarkData()

In [4]:
# Reduce to climate measurement series and delete redunent entries and save to csv sub-database
hplib.ReduceKeymarkData('database_keymark.csv','average')

In [5]:
# Normalize electrical and thermal power from the keymark database to values from setpoint T_in = -7 °C and T_out = 52 °C
hplib.NormalizeKeymarkData('database_keymark_average.csv')

In [6]:
#Get modus like On-Off, Inverter and 2-stages Heatpump
hplib.GetModusforeveryModel('database_keymark_average_normalized.csv')

In [7]:
#Every Heatpump gets a modus depending on its modus and Type
hplib.AssignGroup('database_keymark_average_normalized_modus.csv')

In [ ]:
#Calculate parameters k1-k9
hplib.GetParameters('database_keymark_average_normalized_modus_group.csv')

In [3]:
simulate.getPowerHP(5,50,simulate.getParaeasy('LW 101'))

(10437.040120245001, 3331.87630209069, 0, 3.1324812729980263)